# Исследование надежности заемщиков.

Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

### Шаг 1. Обзор данных

In [52]:
import pandas as pd

In [53]:
data = pd.read_csv('/Users/denisslobodyanyuk/Desktop/датасеты яндекса/data (2).csv')#сохраняем таблицу в переменную

In [54]:
display(data)#просмотр таблицы 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


In [55]:
data.info()#вызовем метод info для просмотра информации о датасете

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Мы просмотрели всю информацию о датафрейме и убедились, что в каждом столбце находится верный тип значений

### Шаг 2.1 Заполнение пропусков

In [56]:
data.isna().sum()#найдем столбцы, где есть пропуски 

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Мы обнаружили, что в столбце `total_income` и `days_employed` есть пропуски. Эти переменные являются количественными. Так как количество пропусков в этих двуз стоблцах одинаковое, мы можем предположить, что у клиента не было ни одного трудового дня, соотвественно и доход также отсутствует.

Узнаем долю пропущенных значений в столбцах `total_income` и `days_employed`:

In [57]:
income_part = data['total_income'].isna().sum() / len(data['total_income'])#считаем  отношение пропущенных значений к длине всего столбца 
print(f'Доля пропущенных значений в total_income составляет {income_part:.1%}')

Доля пропущенных значений в total_income составляет 10.1%


In [58]:
days_part = data['days_employed'].isna().sum() / len(data['days_employed'])#считаем  отношение пропущенных значений к длине всего столбца 
print(f'Доля пропущенных значений в days_employed составляет {days_part:.1%}')

Доля пропущенных значений в days_employed составляет 10.1%


Получается, что доля пропущенных значений одинаковая - `10.1%`.

Теперь заменим пропущенные значения в `total_income` на медианное значение данного столбца. Лучше использовать медианное значение, чем среднее, так как тогда информация будет более объективная.

In [59]:
income_median = data['total_income'].median()#используем метод median
data['total_income'] = data['total_income'].fillna(income_median)#заменим пропущенные значения 

Теперь проверим остались ли пропуски в столбце `total_income`


In [60]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64

Все верно, пропущенных значений в нужном столбце не осталось

### Шаг 2.2 Проверка данных на аномалии и исправления.

Мы уже увидели, что в столбце `days_employed` есть отрицательные значения. Но отрицательного количества дней трудового стажа быть не может, нужно это исправить

In [61]:
data['days_employed'] = abs(data['days_employed'])#применяем функцию модуля 

In [62]:
data.sort_values('days_employed')#проверим количество отрицательных дней стажа

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
17437,1,24.141633,31,среднее,1,женат / замужем,0,F,сотрудник,1,166952.415427,высшее образование
8336,0,24.240695,32,высшее,0,Не женат / не замужем,4,M,сотрудник,0,124115.373655,получение дополнительного образования
6157,2,30.195337,47,среднее,1,гражданский брак,1,M,компаньон,0,231461.185606,свадьба
9683,0,33.520665,43,среднее,1,Не женат / не замужем,4,M,сотрудник,1,128555.897209,приобретение автомобиля
2127,1,34.701045,31,высшее,0,женат / замужем,0,F,компаньон,0,90557.994311,получение образования
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,145017.937533,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,145017.937533,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,145017.937533,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,145017.937533,строительство жилой недвижимости


Столбцов с отрицательным трудовым стажем не осталось 

In [63]:
data.sort_values('days_employed', ascending=False)#

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
6954,0,401755.400475,56,среднее,1,вдовец / вдова,2,F,пенсионер,0,176278.441171,ремонт жилью
10006,0,401715.811749,69,высшее,0,Не женат / не замужем,4,F,пенсионер,0,57390.256908,получение образования
7664,1,401675.093434,61,среднее,1,женат / замужем,0,F,пенсионер,0,126214.519212,операции с жильем
2156,0,401674.466633,60,среднее,1,женат / замужем,0,M,пенсионер,0,325395.724541,автомобили
7794,0,401663.850046,61,среднее,1,гражданский брак,1,F,пенсионер,0,48286.441362,свадьба
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,145017.937533,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,145017.937533,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,145017.937533,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,145017.937533,строительство жилой недвижимости


Также мы видим аномально большие значения в `days_employed`. Например, 400 тысяч дней(это больше 1000 лет). Нужно оставить только реалистичную информацию

In [64]:
days = data[data.loc[:, 'days_employed'] <= 50000]['days_employed']
data.loc[data['days_employed'] > 50000, 'days_employed'] = days.median()
#ставим порог в 50000 дней и большие значения заменяем медианой

In [65]:
data.sort_values('days_employed', ascending=False)#проверим количество отрицательных дней стажа

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
16335,1,18388.949901,61,среднее,1,женат / замужем,0,F,сотрудник,0,186178.934089,операции с недвижимостью
4299,0,17615.563266,61,среднее,1,женат / замужем,0,F,компаньон,0,122560.741753,покупка жилья
7329,0,16593.472817,60,высшее,0,женат / замужем,0,F,сотрудник,0,124697.846781,заняться высшим образованием
17838,0,16264.699501,59,среднее,1,женат / замужем,0,F,сотрудник,0,51238.967133,на покупку автомобиля
16825,0,16119.687737,64,среднее,1,женат / замужем,0,F,сотрудник,0,91527.685995,покупка жилой недвижимости
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,145017.937533,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,145017.937533,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,145017.937533,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,145017.937533,строительство жилой недвижимости


Более того, в дальгнейшей работу обнаружилось, что в столбце `children` присутствуют отрицательные значения и слищком большие(например, 20 детей)

In [66]:
data['children'] = abs(data['children'])#применяем функцию модуля ещё раз 

In [67]:
children = data[data.loc[:, 'children'] <= 5]['children']
data.loc[data['children'] > 5, 'children'] = children.median()
#ставим порог в 5 детей и большие значения заменяем медианой

Мы избавились от аномалий в датасете, удалив слишком большие и отрицательные значения

In [68]:
#проверим на аномалии столбец gender
data = data.query('gender != "XNA"')
data['gender'].value_counts()

F    14236
M     7288
Name: gender, dtype: int64

In [69]:
#теперь столбец dob years
data['dob_years'].value_counts()

#заменим значения 0 на медианное
data['dob_years'] = data['dob_years'].replace(0, data['dob_years'].median())

### Шаг 2.3. Изменение типов данных.

In [70]:
data.info()#еще раз вызываем метод info() и смотрим на тип данных в total_income

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21524 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21524 non-null  int64  
 1   days_employed     19350 non-null  float64
 2   dob_years         21524 non-null  int64  
 3   education         21524 non-null  object 
 4   education_id      21524 non-null  int64  
 5   family_status     21524 non-null  object 
 6   family_status_id  21524 non-null  int64  
 7   gender            21524 non-null  object 
 8   income_type       21524 non-null  object 
 9   debt              21524 non-null  int64  
 10  total_income      21524 non-null  float64
 11  purpose           21524 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


Заменим вещественный тип данных на целочисленный в этом столбце

In [71]:
data['total_income'] = data['total_income'].astype('int')#применяем метод astype

In [72]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21524 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21524 non-null  int64  
 1   days_employed     19350 non-null  float64
 2   dob_years         21524 non-null  int64  
 3   education         21524 non-null  object 
 4   education_id      21524 non-null  int64  
 5   family_status     21524 non-null  object 
 6   family_status_id  21524 non-null  int64  
 7   gender            21524 non-null  object 
 8   income_type       21524 non-null  object 
 9   debt              21524 non-null  int64  
 10  total_income      21524 non-null  int64  
 11  purpose           21524 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.1+ MB


Все верно, тип данных стал верным.

### Шаг 2.4. Удаление дубликатов.

In [73]:
data.head(10)#выведем ещё раз первые 10 строк датасета

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,1630.019381,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


In [74]:
data.duplicated().sum()#найдём количество явных дубликатов

55

In [75]:
data = data.drop_duplicates().reset_index(drop=True)#удаляем явные дубликаты с новыми индексами
data.duplicated().sum()

0

Мы удалили явные дубликаты. Теперь нужно удалить дубликаты с учетом регистра

In [76]:
data['education'] = data['education'].str.lower()
data['family_status'] = data['family_status'].str.lower()
data['gender'] = data['gender'].str.lower()
data['income_type'] = data['income_type'].str.lower()
data['purpose'] = data['purpose'].str.lower()#делаем нижний регистр во всех столбцах, где есть строчные буквы, перезаписав их

In [77]:
data = data.drop_duplicates().reset_index(drop=True)#удаляем дубликаты в данных столбцах

Мы удалили все возможные дубликаты. Теперь проверим нашу работу для каждого из взятых столбцов методом `value_counts`

In [78]:
data['education'].value_counts()

среднее                15171
высшее                  5250
неоконченное высшее      743
начальное                282
ученая степень             6
Name: education, dtype: int64

In [79]:
data['family_status'].value_counts()

женат / замужем          12339
гражданский брак          4149
не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

In [80]:
data['gender'].value_counts()

f    14173
m     7279
Name: gender, dtype: int64

In [81]:
data['income_type'].value_counts()

сотрудник          11083
компаньон           5077
пенсионер           3829
госслужащий         1457
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

In [82]:
data['purpose'].value_counts()

свадьба                                   791
на проведение свадьбы                     767
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      620
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

Мы удалили сначала явные дубликаты, затем все дубликаты, приведенные к нижнему регистру. В итоге мы видим, что дубликатов в столбцах не осталось

Такие дубликаты чаще всего могут появляться при заполнение данных сотрудниками, которые могут "не заметить" такие опечатки



### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

In [83]:
data_education = data.pivot_table(index='education', values='education_id')
display(data_education)#создаёем первый датафрейм по education

,education_id
education,
высшее,0
начальное,3
неоконченное высшее,2
среднее,1
ученая степень,4


In [84]:
data_family = data.pivot_table(index='family_status', values='family_status_id')
data_family['family_status_id'] = data_family['family_status_id'].astype('int')
display(data_family)#создаем второй датафрейм по family_status

,family_status_id
family_status,
в разводе,3
вдовец / вдова,2
гражданский брак,1
женат / замужем,0
не женат / не замужем,4


In [85]:
data.drop(columns = ['education', 'family_status'], axis = 1)# удаляем два столбца из исходного датафрейма

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,0,0,f,сотрудник,0,253875,покупка жилья
1,1,4024.803754,36,1,0,f,сотрудник,0,112080,приобретение автомобиля
2,0,5623.422610,33,1,0,m,сотрудник,0,145885,покупка жилья
3,3,4124.747207,32,1,0,m,сотрудник,0,267628,дополнительное образование
4,0,1630.019381,53,1,1,f,пенсионер,0,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...
21447,1,4529.316663,43,1,1,f,компаньон,0,224791,операции с жильем
21448,0,1630.019381,67,1,0,f,пенсионер,0,155999,сделка с автомобилем
21449,1,2113.346888,38,1,1,m,сотрудник,1,89672,недвижимость
21450,3,3112.481705,38,1,0,m,сотрудник,1,244093,на покупку своего автомобиля


В этом пункте мы создали два новых датафрейма и удалили столбцы `education` и `family_status` из исходного

### Шаг 2.6. Категоризация дохода.

In [86]:
def income_group(number):
    if number <= 30000:
        return 'E'
    if number <= 50000:
        return 'D'
    if number <= 200000:
        return 'C'
    if number <= 1000000:
        return 'B'
    return 'A'#напишем функцию категоризации доходов

In [87]:
data['total_income_category'] = data['total_income'].apply(income_group)
#применим функцию к столбцу total_income

In [88]:
data.head()#выводим первые пять строк для проверки

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875,покупка жилья,B
1,1,4024.803754,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623.422610,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885,покупка жилья,C
3,3,4124.747207,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628,дополнительное образование,B
4,0,1630.019381,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616,сыграть свадьбу,C


### Шаг 2.7. Категоризация целей кредита.

In [89]:
def purpose_group(purpose):
    if 'авто' in purpose:
        return 'операции с автомобилем'
    if 'недвиж' in purpose or 'жиль' in purpose:
        return 'операции с недвижимостью'
    if 'свадьб' in purpose:
        return 'проведение свадьбы'
    if 'образован' in purpose:
        return 'получение образования'
#напишем функцию категоризации целей кредита

In [90]:
#data['purpose_category'] = data['purpose'].apply(purpose_group)#применяем функцию
#мы увидели ошибку при вызове функции

Мы увидели, что в столбце `purpose` содержатся данные типа `float`. Нужно это изменить

In [91]:
data['purpose'] = data['purpose'].astype('str')#меняем тип данных на str

In [92]:
data['purpose_category'] = data['purpose'].apply(purpose_group)
display(data)#применяем функцию ещё раз и выводим датасет на экран

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024.803754,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623.422610,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124.747207,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,1630.019381,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21447,1,4529.316663,43,среднее,1,гражданский брак,1,f,компаньон,0,224791,операции с жильем,B,операции с недвижимостью
21448,0,1630.019381,67,среднее,1,женат / замужем,0,f,пенсионер,0,155999,сделка с автомобилем,C,операции с автомобилем
21449,1,2113.346888,38,среднее,1,гражданский брак,1,m,сотрудник,1,89672,недвижимость,C,операции с недвижимостью
21450,3,3112.481705,38,среднее,1,женат / замужем,0,m,сотрудник,1,244093,на покупку своего автомобиля,B,операции с автомобилем


Мы создали верную функцию для категризации целей кредита и добавили новый столбец - `purpose_category`

### Ответы на вопросы.

##### Вопрос 1: 

Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [93]:
#создадим сводную таблицу для этого вопроса
borrowers = data.pivot_table(index = 'children', values='debt', aggfunc = ('mean', lambda X:
                                                                       X.count()))

#создадим функцию для перевода соотношения в проценты 
def percent(value):
    return "{0:.2%}".format(value)

# изменим название столбцов и применяем функцию 
borrowers = borrowers.rename(columns={'<lambda_0>': 'amount', 'mean': 'percentage'})
borrowers['percentage'] = borrowers['percentage'].apply(percent)
borrowers.sort_values(by='amount', ascending=False)


,amount,percentage
children,,
0,14165.0,7.56%
1,4855.0,9.17%
2,2052.0,9.45%
3,330.0,8.18%
4,41.0,9.76%
5,9.0,0.00%


##### Вывод 1:

Из данной таблицы можно сделать вывод, что зависимость опредленно присутствует. Самые ответсвенные семьи по выплате кредита являются те, у которых нет детей. Затем идут семьи с 3 детьми, а уже после те, у кого 1, 2 или 4 ребёнка.  

##### Вопрос 2: 

Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [94]:
#создадим свобдную таблицу
family = data.pivot_table(index = 'family_status', values='debt', aggfunc = ('mean', lambda X:
                                                                       X.count()))

# затем снова применим функцию для процентов и переименуем столбцы
family['mean'] = family['mean'].apply(percent)
family = family.rename(columns={"<lambda_0>": "amount", "mean": "percentage"})
family.sort_values(by='amount', ascending=False)

,amount,percentage
family_status,,
женат / замужем,12339.0,7.55%
гражданский брак,4149.0,9.35%
не женат / не замужем,2810.0,9.75%
в разводе,1195.0,7.11%
вдовец / вдова,959.0,6.57%


##### Вывод 2:

Как мы видим из таблицы, зависимость явно присутсвует. Отвественее всего к возврату кредита относятся вдовцы - всего `6.55%`. А самыми неответсвенными клиентами являются те, кто не состоят в браке - `10.49%`.

##### Вопрос 3: 

Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [95]:
#применим такие же действия для столбца total_income_category
income = data.pivot_table(index = 'total_income_category', values='debt', aggfunc = ('mean', lambda X:
                                                                       X.count()))

income['mean'] = income['mean'].apply(percent)
income = income.rename(columns={"<lambda_0>": 'amount', 'mean': 'percentage'})
income.sort_values(by='amount', ascending=False)

,amount,percentage
total_income_category,,
C,16015.0,8.49%
B,5040.0,7.06%
D,350.0,6.00%
A,25.0,8.00%
E,22.0,9.09%


##### Вывод 3:

Здесь также присутствует зависимость. Самый выоский средний показатель имеют клиенты с низким доходом, но в этой строке у нас всего 8 записей, что является очень маленькой частью. Самый низкий показатель имеют клиенты с высоким уровнем дохода.

##### Вопрос 4: 

Как разные цели кредита влияют на его возврат в срок?

In [96]:
#применим такие же действия для столбца purpose_category
purpose = data.pivot_table(index = 'purpose_category', values='debt', aggfunc = ('mean', lambda X:
                                                                       X.count()))

purpose['mean'] = purpose['mean'].apply(percent)
purpose = purpose.rename(columns={"<lambda_0>": 'amount', 'mean': 'percentage'})
purpose.sort_values(by='amount', ascending=False)

,amount,percentage
purpose_category,,
операции с недвижимостью,10810.0,7.23%
операции с автомобилем,4306.0,9.36%
получение образования,4013.0,9.22%
проведение свадьбы,2323.0,8.01%


##### Вывод 4:

Лучшее значение у тех, кто собирается покупать недвижимость - `7.73%`. А самый худший показатель у клиентов с целью покупки авто - `10.01%`.

## Общий вывод:

В данном проекте мы увидели, что все категории имеют зависимость с выплатой кредита в срок. Мы поняли, что семейной положение, уровень дохода, количество детей и цели взятия сильно влияют на выплату кредита и нужно их все учитывать.

#### Категория детей: 

1. Самые надежные клиенты - те, у кого нет детей

2. Самые безответсвенные - семьи с 4 детьми 

#### По семейному статусу: 

1. Самые надежные клиенты - вдовцы

2. Самые безответсвенные - люди, не состоявшие в браке

#### По уровню дохода: 

1. Самые надежные клиенты - те, у кого низкий доход

2. Самые безответсвенные - у кого доход выше среднего

#### По цели взятия кредита: 

1. Самые надежные клиенты - покупка квартиры

2. Самые безответсвенные - покупка авто  